In [1]:
# Show Data Profile
def show_data_profile(df):
  Data_dict = pd.DataFrame(df.dtypes)
  Data_dict = Data_dict.rename(columns={0: 'Data_Type'})
  Data_dict['Missing'] = df.isnull().sum()
  Data_dict['Missing_Pct'] = round((df.isnull().sum()/df.count())*100, 2)
  Data_dict['Unique'] = df.nunique()
  Data_dict['Count'] = df.count()
  Data_dict=Data_dict.reset_index()
  desc = df.describe(include='all').T
  desc = desc.drop(['count', 'unique'], axis=1)
  desc = desc.reset_index()
  Data_dict2 = pd.merge(Data_dict,desc,how='left', on=['index'])
  Data_dict2 = Data_dict2.fillna('-')#.sort_values(by=['Data_Type','Missing_Pct'],ascending=[True,False])
  Data_dict2 = Data_dict2.reset_index(drop=True).rename(columns={'index':'Features'})
  return Data_dict2

In [2]:
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)
warnings.filterwarnings("ignore",category=RuntimeWarning)


import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('mode.chained_assignment', None)

import re
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns



#DATA PREPARATION

In [20]:
#Load Data
df = pd.read_csv("https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/all/online-retail-dataset.csv", sep=",",encoding='latin1',dtype={'CustomerID': str} )

In [21]:
df.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


**Attribute Information:**

**InvoiceNo**: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.  

**StockCode**: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.  

**Description**: Product (item) name. Nominal.  

**Quantity**: The quantities of each product (item) per transaction. Numeric.  

**InvoiceDate**: Invice Date and time. Numeric, the day and time when each transaction was generated.  

**UnitPrice**: Unit price. Numeric, Product price per unit in sterling.  

**CustomerID**: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.  

**Country**: Country name. Nominal, the name of the country where each customer resides.

In [22]:
#Rename column replacing whitespace with underscore
df.rename(columns= lambda x : x.replace(' ','_') if ' ' in x else x, inplace=True)
#Remove row with all columns are NA
df = df.dropna(how='all')

In [23]:
show_data_profile(df)

,Features,Data_Type,Missing,Missing_Pct,Unique,Count,top,freq,mean,std,min,25%,50%,75%,max
0,InvoiceNo,object,0,0.00,25900,541909,573585,1114,-,-,-,-,-,-,-
1,StockCode,object,0,0.00,4070,541909,85123A,2313,-,-,-,-,-,-,-
2,Description,object,1454,0.27,4223,540455,WHITE HANGING HEART T-LIGHT HOLDER,2369,-,-,-,-,-,-,-
3,Quantity,int64,0,0.00,722,541909,-,-,9.55225,218.081,-80995,1,3,10,80995
4,InvoiceDate,object,0,0.00,23260,541909,10/31/2011 14:41,1114,-,-,-,-,-,-,-
5,UnitPrice,float64,0,0.00,1630,541909,-,-,4.61111,96.7599,-11062.1,1.25,2.08,4.13,38970
6,CustomerID,object,135080,33.20,4372,406829,17841,7983,-,-,-,-,-,-,-
7,Country,object,0,0.00,38,541909,United Kingdom,495478,-,-,-,-,-,-,-


In [24]:
#Convert datetime column to datetime format
from datetime import datetime
df['InvoiceDate'] = df['InvoiceDate'].apply(lambda x:  datetime.strptime(x, '%m/%d/%Y %H:%M'))

In [25]:
#Split column to alphabet and numeric
df['Status']=df['InvoiceNo'].apply(lambda x : ''.join(re.findall(r"[a-zA-Z]",str(x))))
df['Status']=df['Status'].replace('','S')
df['InvoiceNo']=df['InvoiceNo'].apply(lambda x : ''.join(re.findall(r"\d+",str(x))))

In [26]:
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Status
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,S
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,S
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,S


In [27]:
df['Status'].unique()

array(['S', 'C', 'A'], dtype=object)

In [32]:
dF = df[df['Status']=='S']

## Feature Engineering

In [33]:
#Add Revenue column
df['Revenue']=df['Quantity']*df['UnitPrice']

In [34]:
#Add splitted datetime column
df['yearmonth']=df['InvoiceDate'].dt.strftime('%Y%m')
df['year']=df['InvoiceDate'].dt.strftime('%Y')
df['month']=df['InvoiceDate'].dt.strftime('%m')
df['date']=df['InvoiceDate'].dt.strftime('%d')
df['hour']=df['InvoiceDate'].dt.strftime('%H')
df['dayofweek']=df['InvoiceDate'].dt.strftime('%A') 
df['indexofweek']=(df['InvoiceDate'].dt.day - 1) // 7 + 1 #Index of week in a month (1-5)
df['quarter']=df['InvoiceDate'].dt.quarter # 1 = Jan-Mar, 2 = Apr-Jun, dst


In [35]:
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Status,Revenue,yearmonth,year,month,date,hour,dayofweek,indexofweek,quarter
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,S,15.30,201012,2010,12,01,08,Wednesday,1,4
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,S,20.34,201012,2010,12,01,08,Wednesday,1,4
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,S,22.00,201012,2010,12,01,08,Wednesday,1,4


In [36]:
#Convert data type
df['indexofweek']=df['indexofweek'].astype(str)
df['quarter']=df['quarter'].astype(str)

In [37]:
#Add first transaction
df['First_InvoiceDate']=df.groupby('CustomerID')['InvoiceDate'].transform(min)
#Add last transaction
df['Last_InvoiceDate']=df.groupby('CustomerID')['InvoiceDate'].transform(max)

In [38]:
#Add num of days between current and first trx
df['days_current_first'] = (df['InvoiceDate']-df['First_InvoiceDate']).dt.days

In [39]:
#Add num of days between last and first trx
df['days_last_first'] = (df['Last_InvoiceDate']-df['First_InvoiceDate']).dt.days

In [40]:
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Status,Revenue,yearmonth,year,month,date,hour,dayofweek,indexofweek,quarter,First_InvoiceDate,Last_InvoiceDate,days_current_first,days_last_first
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,S,15.30,201012,2010,12,01,08,Wednesday,1,4,2010-12-01 08:26:00,2011-02-10 14:38:00,0.0,71.0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,S,20.34,201012,2010,12,01,08,Wednesday,1,4,2010-12-01 08:26:00,2011-02-10 14:38:00,0.0,71.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,S,22.00,201012,2010,12,01,08,Wednesday,1,4,2010-12-01 08:26:00,2011-02-10 14:38:00,0.0,71.0


In [41]:
show_data_profile(df)

,Features,Data_Type,Missing,Missing_Pct,Unique,Count,top,freq,first,last,mean,std,min,25%,50%,75%,max
0,InvoiceNo,object,0,0.00,25900,541909,573585,1114,-,-,-,-,-,-,-,-,-
1,StockCode,object,0,0.00,4070,541909,85123A,2313,-,-,-,-,-,-,-,-,-
2,Description,object,1454,0.27,4223,540455,WHITE HANGING HEART T-LIGHT HOLDER,2369,-,-,-,-,-,-,-,-,-
3,Quantity,int64,0,0.00,722,541909,-,-,-,-,9.55225,218.081,-80995,1,3,10,80995
4,InvoiceDate,datetime64[ns],0,0.00,23260,541909,2011-10-31 14:41:00,1114,2010-12-01 08:26:00,2011-12-09 12:50:00,-,-,-,-,-,-,-
5,UnitPrice,float64,0,0.00,1630,541909,-,-,-,-,4.61111,96.7599,-11062.1,1.25,2.08,4.13,38970
6,CustomerID,object,135080,33.20,4372,406829,17841,7983,-,-,-,-,-,-,-,-,-
7,Country,object,0,0.00,38,541909,United Kingdom,495478,-,-,-,-,-,-,-,-,-
8,Status,object,0,0.00,3,541909,S,532618,-,-,-,-,-,-,-,-,-
9,Revenue,float64,0,0.00,6204,541909,-,-,-,-,17.9878,378.811,-168470,3.4,9.75,17.4,168470


## Data Cleaning

In [42]:
#Remove row with CustomerID=NA
df = df[~df['CustomerID'].isna()]
df['Description'] = df['Description'].fillna('')

In [43]:
#Remove row with Negative Quantity
df = df[df['Quantity'] >0]

In [44]:
show_data_profile(df)

,Features,Data_Type,Missing,Missing_Pct,Unique,Count,top,freq,first,last,mean,std,min,25%,50%,75%,max
0,InvoiceNo,object,0,0.0,18536,397924,576339,542,-,-,-,-,-,-,-,-,-
1,StockCode,object,0,0.0,3665,397924,85123A,2035,-,-,-,-,-,-,-,-,-
2,Description,object,0,0.0,3877,397924,WHITE HANGING HEART T-LIGHT HOLDER,2028,-,-,-,-,-,-,-,-,-
3,Quantity,int64,0,0.0,302,397924,-,-,-,-,13.0218,180.42,1,2,6,12,80995
4,InvoiceDate,datetime64[ns],0,0.0,17286,397924,2011-11-14 15:27:00,542,2010-12-01 08:26:00,2011-12-09 12:50:00,-,-,-,-,-,-,-
5,UnitPrice,float64,0,0.0,441,397924,-,-,-,-,3.11617,22.0968,0,1.25,1.95,3.75,8142.75
6,CustomerID,object,0,0.0,4339,397924,17841,7847,-,-,-,-,-,-,-,-,-
7,Country,object,0,0.0,37,397924,United Kingdom,354345,-,-,-,-,-,-,-,-,-
8,Status,object,0,0.0,1,397924,S,397924,-,-,-,-,-,-,-,-,-
9,Revenue,float64,0,0.0,2940,397924,-,-,-,-,22.3947,309.056,0,4.68,11.8,19.8,168470


## Transaction aggregation by customerID

In [45]:
max_date=df['InvoiceDate'].max() + pd.to_timedelta(1, unit='d')
max_date

Timestamp('2011-12-10 12:50:00')

In [47]:
df_customer = df.groupby(['CustomerID']).aggregate({'Quantity':sum,'Revenue':sum,'InvoiceNo': pd.Series.nunique,'StockCode': pd.Series.nunique,'InvoiceDate':[min,max]}).reset_index()
df_customer.columns=['CustomerID','Quantity','Revenue','Count_Invoice','Count_StockCode','First_Invoice','Last_Invoice']
df_customer['diff_last_first']=(df_customer['Last_Invoice']-df_customer['First_Invoice']).dt.days
df_customer['diff_max_last']=(max_date-df_customer['Last_Invoice']).dt.days
df_customer.sort_values(by='Revenue',ascending=False)

,CustomerID,Quantity,Revenue,Count_Invoice,Count_StockCode,First_Invoice,Last_Invoice,diff_last_first,diff_max_last
1690,14646,197491,280206.02,74,701,2010-12-20 10:09:00,2011-12-08 12:12:00,353,2
4202,18102,64124,259657.30,60,150,2010-12-07 16:42:00,2011-12-09 11:50:00,366,1
3729,17450,69993,194550.79,46,124,2010-12-07 09:23:00,2011-12-01 13:29:00,359,8
3009,16446,80997,168472.50,2,3,2011-05-18 09:52:00,2011-12-09 09:15:00,204,1
1880,14911,80515,143825.06,201,1787,2010-12-01 14:05:00,2011-12-08 15:54:00,372,1
...,...,...,...,...,...,...,...,...,...
4099,17956,1,12.75,1,1,2011-04-04 13:47:00,2011-04-04 13:47:00,0,249
3015,16454,3,6.90,2,2,2011-10-06 16:57:00,2011-10-26 11:40:00,19,45
1794,14792,2,6.20,1,2,2011-10-07 09:19:00,2011-10-07 09:19:00,0,64
3218,16738,3,3.75,1,1,2011-02-15 09:46:00,2011-02-15 09:46:00,0,298


#CUSTOMER SEGMENTATION (RFM)

In [48]:
df_rfm = df_customer[['CustomerID','diff_max_last','Count_Invoice','Revenue']]
df_rfm.columns=['CustomerID','Recency','Frequency','Monetary']
df_rfm.head(3)

,CustomerID,Recency,Frequency,Monetary
0,12346,326,1,77183.60
1,12347,2,7,4310.00
2,12348,75,4,1797.24


In [49]:
df_rfm.describe()

,Recency,Frequency,Monetary
count,4339.000000,4339.000000,4339.000000
mean,92.518322,4.271952,2053.793018
std,100.009747,7.705493,8988.248381
min,1.000000,1.000000,0.000000
25%,18.000000,1.000000,307.245000
50%,51.000000,2.000000,674.450000
75%,142.000000,5.000000,1661.640000
max,374.000000,210.000000,280206.020000


In [50]:
bin = [0,1,100,200,300,400]
bin_label = [5,4,3,2,1]
df_rfm['R'] = pd.cut(df_rfm['Recency'], bins=bin, labels=bin_label)

In [51]:
bin = [0,1,10,50,100,250]
bin_label = [1,2,3,4,5]
df_rfm['F'] = pd.cut(df_rfm['Frequency'], bins=bin, labels=bin_label)

In [52]:
bin = [-1,1,50000,100000,200000,300000]
bin_label = [1,2,3,4,5]
df_rfm['M'] = pd.cut(df_rfm['Monetary'], bins=bin, labels=bin_label)

In [53]:
df_rfm.head(3)

,CustomerID,Recency,Frequency,Monetary,R,F,M
0,12346,326,1,77183.60,1,1,3
1,12347,2,7,4310.00,4,2,2
2,12348,75,4,1797.24,4,2,2


In [54]:
show_data_profile(df_rfm)

,Features,Data_Type,Missing,Missing_Pct,Unique,Count,top,freq,mean,std,min,25%,50%,75%,max
0,CustomerID,object,0,0.0,4339,4339,16160,1,-,-,-,-,-,-,-
1,Recency,int64,0,0.0,349,4339,-,-,92.5183,100.01,1,18,51,142,374
2,Frequency,int64,0,0.0,59,4339,-,-,4.27195,7.70549,1,1,2,5,210
3,Monetary,float64,0,0.0,4285,4339,-,-,2053.79,8988.25,0,307.245,674.45,1661.64,280206
4,R,category,0,0.0,5,4339,4,2887,-,-,-,-,-,-,-
5,F,category,0,0.0,5,4339,2,2508,-,-,-,-,-,-,-
6,M,category,0,0.0,5,4339,2,4318,-,-,-,-,-,-,-


In [55]:
funct = lambda x : str(x['R'])+str(x['F'])+str(x['M'])
df_rfm['RFM_Segment']=df_rfm.apply(funct,axis=1)

In [ ]:
df_rfm.head(3)

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM_Segment
1690,14646,2,74,280206.02,4,4,5,445
4202,18102,1,60,259657.30,5,4,5,545
3729,17450,8,46,194550.79,4,3,4,434


### Score based on sum

https://towardsdatascience.com/recency-frequency-monetary-model-with-python-and-how-sephora-uses-it-to-optimize-their-google-d6a0707c5f17

In [56]:
df_rfm1=df_rfm.copy()

In [57]:
df_rfm1['RFM_Score'] = df_rfm1[['R','F','M']].sum(axis=1)

In [58]:
# Define rfm_level function
def rfm_level(df):
    if df['RFM_Score'] >= 9:
        return 'Can\'t Loose Them'
    elif ((df['RFM_Score'] >= 8) and (df['RFM_Score'] < 9)):
        return 'Champions'
    elif ((df['RFM_Score'] >= 7) and (df['RFM_Score'] < 8)):
        return 'Loyal'
    elif ((df['RFM_Score'] >= 6) and (df['RFM_Score'] < 7)):
        return 'Potential'
    elif ((df['RFM_Score'] >= 5) and (df['RFM_Score'] < 6)):
        return 'Promising'
    elif ((df['RFM_Score'] >= 4) and (df['RFM_Score'] < 5)):
        return 'Needs Attention'
    else:
        return 'Require Activation'
# Create a new variable RFM_Level
df_rfm1['RFM_Level'] = df_rfm1.apply(rfm_level, axis=1)
# Print the header with top 5 rows to the console
df_rfm1.head()

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM_Segment,RFM_Score,RFM_Level
0,12346,326,1,77183.60,1,1,3,113,5,Promising
1,12347,2,7,4310.00,4,2,2,422,8,Champions
2,12348,75,4,1797.24,4,2,2,422,8,Champions
3,12349,19,1,1757.55,4,1,2,412,7,Loyal
4,12350,310,1,334.40,1,1,2,112,4,Needs Attention


In [59]:
#Finally, we can then group our customers by their RFM level.
# Calculate average values for each RFM_Level, and return a size of each segment 
rfm_level_agg = df_rfm1.groupby('RFM_Level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'count']
}).round(1)
# Print the aggregated dataset
rfm_level_agg

Recency Frequency Monetary      
                    mean      mean     mean count
RFM_Level                                        
Can't Loose Them    11.3      19.1  12125.6   382
Champions           33.7       4.2   1595.7  1935
Loyal               82.2       1.7    628.5  1022
Needs Attention    342.3       1.0    304.2   225
Potential          181.6       1.6    680.0   391
Promising          260.1       1.2    560.3   384

### Score based on Segment Mapping

https://docs.exponea.com/docs/rfm-segmentation

<table><thead><tr><th style="text-align: left;">Segment</th><th style="text-align: left;">Scores</th></tr></thead><tbody><tr><td style="text-align: left;">Champions</td><td style="text-align: left;">555, 554, 544, 545, 454, 455, 445</td></tr><tr><td style="text-align: left;">Loyal</td><td style="text-align: left;">543, 444, 435, 355, 354, 345, 344, 335</td></tr><tr><td style="text-align: left;">Potential Loyalist</td><td style="text-align: left;">553, 551,552, 541,542, 533, 532, 531, 452, 451, 442, 441, 431, 453, 433, 432, 423, 353, 352, 351, 342, 341, 333, 323</td></tr><tr><td style="text-align: left;">Recent Customers</td><td style="text-align: left;">512, 511, 422, 421, 412, 411, 311</td></tr><tr><td style="text-align: left;">Promising</td><td style="text-align: left;">525, 524, 523, 522, 521, 515, 514, 513, 425,424, 413,414,415, 315, 314, 313</td></tr><tr><td style="text-align: left;">Need Attention</td><td style="text-align: left;">535, 534, 443, 434, 343, 334, 325, 324</td></tr><tr><td style="text-align: left;">About To Sleep</td><td style="text-align: left;">331, 321, 312, 221, 213, 231, 241, 251</td></tr><tr><td style="text-align: left;">At Risk</td><td style="text-align: left;">255, 254, 245, 244, 253, 252, 243, 242, 235, 234, 225, 224, 153, 152, 145, 143, 142, 135, 134, 133, 125, 124</td></tr><tr><td style="text-align: left;">Cannot Lose Them</td><td style="text-align: left;">155, 154, 144, 214,215,115, 114, 113</td></tr><tr><td style="text-align: left;">Hibernating customers</td><td style="text-align: left;">332, 322, 231, 241, 251, 233, 232, 223, 222, 132, 123, 122, 212, 211</td></tr><tr><td style="text-align: left;">Lost customers</td><td style="text-align: left;">111, 112, 121, 131,141,151</td></tr></tbody></table>

<table class="wide"><thead><tr><th>Customer Segment</th><th>Activity</th><th>Actionable Tip</th></tr></thead><tbody><tr><td>Champions</td><td>Bought recently, buy often and spend the most!</td><td>Reward them. Can be early adopters for new products. Will promote your brand.</td></tr><tr><td>Loyal Customers</td><td>Spend good money with us often. Responsive to promotions.</td><td>Upsell higher value products. Ask for reviews. Engage them.</td></tr><tr><td>Potential Loyalist</td><td>Recent customers, but spent a good amount and bought more than once.</td><td>Offer membership / loyalty program, recommend other products.</td></tr><tr><td>Recent Customers</td><td>Bought most recently, but not often.</td><td>Provide on-boarding support, give them early success, start building relationship.</td></tr><tr><td>Promising</td><td>Recent shoppers, but haven’t spent much.</td><td>Create brand awareness, offer free trials</td></tr><tr><td>Customers Needing Attention</td><td>Above average recency, frequency and monetary values. May not have bought very recently though.</td><td>Make limited time offers, Recommend based on past purchases. Reactivate them.</td></tr><tr><td>About To Sleep</td><td>Below average recency, frequency and monetary values. Will lose them if not reactivated.</td><td>Share valuable resources, recommend popular products / renewals at discount, reconnect with them.</td></tr><tr><td>At Risk</td><td>Spent big money and purchased often. But long time ago. Need to bring them back!</td><td>Send personalized emails to reconnect, offer renewals, provide helpful resources.</td></tr><tr><td>Can’t Lose Them</td><td>Made biggest purchases, and often. But haven’t returned for a long time.</td><td>Win them back via renewals or newer products, don’t lose them to competition, talk to them.</td></tr><tr><td>Hibernating</td><td>Last purchase was long back, low spenders and low number of orders.</td><td>Offer other relevant products and special discounts. Recreate brand value.</td></tr><tr><td>Lost</td><td>Lowest recency, frequency and monetary scores.</td><td>Revive interest with reach out campaign, ignore otherwise.</td></tr></tbody></table>

In [60]:
import pandas as pd
from io import StringIO

In [61]:
data_string = """Segment;Scores
Champions;555, 554, 544, 545, 454, 455, 445
Loyal;543, 444, 435, 355, 354, 345, 344, 335
Potential Loyalist;553, 551,552, 541,542, 533, 532, 531, 452, 451, 442, 441, 431, 453, 433, 432, 423, 353, 352, 351, 342, 341, 333, 323
Recent Customers;512, 511, 422, 421, 412, 411, 311
Promising;525, 524, 523, 522, 521, 515, 514, 513, 425,424, 413,414,415, 315, 314, 313
Need Attention;535, 534, 443, 434, 343, 334, 325, 324
About To Sleep;331, 321, 312, 221, 213, 231, 241, 251
At Risk;255, 254, 245, 244, 253, 252, 243, 242, 235, 234, 225, 224, 153, 152, 145, 143, 142, 135, 134, 133, 125, 124
Cannot Lose Them;155, 154, 144, 214,215,115, 114, 113
Hibernating customers;332, 322, 231, 241, 251, 233, 232, 223, 222, 132, 123, 122, 212, 211
Lost customers;111, 112, 121, 131,141,151"""

In [62]:
data = StringIO(data_string)
df_ref = pd.read_csv(data, sep=";")
df_ref

,Segment,Scores
0,Champions,"555, 554, 544, 545, 454, 455, 445"
1,Loyal,"543, 444, 435, 355, 354, 345, 344, 335"
2,Potential Loyalist,"553, 551,552, 541,542, 533, 532, 531, 452, 451..."
3,Recent Customers,"512, 511, 422, 421, 412, 411, 311"
4,Promising,"525, 524, 523, 522, 521, 515, 514, 513, 425,42..."
5,Need Attention,"535, 534, 443, 434, 343, 334, 325, 324"
6,About To Sleep,"331, 321, 312, 221, 213, 231, 241, 251"
7,At Risk,"255, 254, 245, 244, 253, 252, 243, 242, 235, 2..."
8,Cannot Lose Them,"155, 154, 144, 214,215,115, 114, 113"
9,Hibernating customers,"332, 322, 231, 241, 251, 233, 232, 223, 222, 1..."


In [63]:
df_ref['Segment'] = df_ref['Segment'].astype(str).apply(lambda x: x.strip()) #Remove whitespace in left and right string
df_ref['Scores'] = df_ref['Scores'].str.replace(" ","")#Remove whitespace in all string
df_ref

,Segment,Scores
0,Champions,"555,554,544,545,454,455,445"
1,Loyal,"543,444,435,355,354,345,344,335"
2,Potential Loyalist,"553,551,552,541,542,533,532,531,452,451,442,44..."
3,Recent Customers,"512,511,422,421,412,411,311"
4,Promising,"525,524,523,522,521,515,514,513,425,424,413,41..."
5,Need Attention,"535,534,443,434,343,334,325,324"
6,About To Sleep,"331,321,312,221,213,231,241,251"
7,At Risk,"255,254,245,244,253,252,243,242,235,234,225,22..."
8,Cannot Lose Them,"155,154,144,214,215,115,114,113"
9,Hibernating customers,"332,322,231,241,251,233,232,223,222,132,123,12..."


In [64]:
ref=[]
for i in df_ref.iterrows():
  segment = i[1][0]
  scores = i[1][1]
  scores_split=scores.split(',')
  for score in scores_split:
    ref.append([score,segment])

ref_dict=dict(ref)
ref_dict

{'111': 'Lost customers',
 '112': 'Lost customers',
 '113': 'Cannot Lose Them',
 '114': 'Cannot Lose Them',
 '115': 'Cannot Lose Them',
 '121': 'Lost customers',
 '122': 'Hibernating customers',
 '123': 'Hibernating customers',
 '124': 'At Risk',
 '125': 'At Risk',
 '131': 'Lost customers',
 '132': 'Hibernating customers',
 '133': 'At Risk',
 '134': 'At Risk',
 '135': 'At Risk',
 '141': 'Lost customers',
 '142': 'At Risk',
 '143': 'At Risk',
 '144': 'Cannot Lose Them',
 '145': 'At Risk',
 '151': 'Lost customers',
 '152': 'At Risk',
 '153': 'At Risk',
 '154': 'Cannot Lose Them',
 '155': 'Cannot Lose Them',
 '211': 'Hibernating customers',
 '212': 'Hibernating customers',
 '213': 'About To Sleep',
 '214': 'Cannot Lose Them',
 '215': 'Cannot Lose Them',
 '221': 'About To Sleep',
 '222': 'Hibernating customers',
 '223': 'Hibernating customers',
 '224': 'At Risk',
 '225': 'At Risk',
 '231': 'Hibernating customers',
 '232': 'Hibernating customers',
 '233': 'Hibernating customers',
 '234': 'A

In [65]:
df_rfm2=df_rfm.copy()
df_rfm2.head(3)

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM_Segment
0,12346,326,1,77183.60,1,1,3,113
1,12347,2,7,4310.00,4,2,2,422
2,12348,75,4,1797.24,4,2,2,422


In [66]:
df_rfm2['RFM_Segment']=df_rfm2['RFM_Segment'].astype(str)

In [67]:
df_rfm2['RFM_Level']=df_rfm2['RFM_Segment'].apply(lambda x : ref_dict[x])

In [68]:
df_rfm2

,CustomerID,Recency,Frequency,Monetary,R,F,M,RFM_Segment,RFM_Level
0,12346,326,1,77183.60,1,1,3,113,Cannot Lose Them
1,12347,2,7,4310.00,4,2,2,422,Recent Customers
2,12348,75,4,1797.24,4,2,2,422,Recent Customers
3,12349,19,1,1757.55,4,1,2,412,Recent Customers
4,12350,310,1,334.40,1,1,2,112,Lost customers
...,...,...,...,...,...,...,...,...,...
4334,18280,278,1,180.60,2,1,2,212,Hibernating customers
4335,18281,181,1,80.82,3,1,2,312,About To Sleep
4336,18282,8,2,178.05,4,2,2,422,Recent Customers
4337,18283,4,16,2094.88,4,3,2,432,Potential Loyalist


In [69]:
df_rfm2.groupby('RFM_Level').aggregate({'CustomerID':pd.Series.nunique,'Recency':np.mean,'Frequency':np.mean,'Monetary':np.mean}).astype(int).sort_values(by='Monetary')

,CustomerID,Recency,Frequency,Monetary
RFM_Level,,,,
Lost customers,225,342,1,304
About To Sleep,256,151,1,374
Hibernating customers,877,210,2,800
Recent Customers,2598,37,3,1284
Promising,51,1,5,5188
Potential Loyalist,322,12,19,9460
Cannot Lose Them,1,326,1,77183
Loyal,2,5,73,89073
Need Attention,4,18,56,114828
